In [ ]:
#!/usr/bin/env python
import gi
gi.require_version('GstBase', '1.0')
gi.require_version('Gtk', '3.0')
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, Gtk, GstBase
#GObject.threads_init()
Gst.init(None)

#
# Simple Sink element created entirely in python
#
class MySink(GstBase.BaseSink):
    __gstmetadata__ = ('CustomSink','Sink', \
                      'Custom test sink element', 'Edward Hervey')

    __gsttemplates__ = Gst.PadTemplate.new("sink",
                                           Gst.PadDirection.SINK,
                                           Gst.PadPresence.ALWAYS,
                                           Gst.Caps.new_any())

    def do_render(self, buffer):
        Gst.info("timestamp(buffer):%s" % (Gst.TIME_ARGS(buffer.pts)))
        return Gst.FlowReturn.OK

GObject.type_register(MySink)
__gstelementfactory__ = ("mysink", Gst.Rank.NONE, MySink)


pipeline = Gst.Pipeline()

autoaudiosrc = Gst.ElementFactory.make("autoaudiosrc", "autoaudiosrc")
audioconvert = Gst.ElementFactory.make("audioconvert", "audioconvert")
vorbisenc = Gst.ElementFactory.make("vorbisenc", "vorbisenc")
oggmux = Gst.ElementFactory.make("oggmux", "oggmux")
filesink = MySink() #Gst.ElementFactory.make("mysink", "mysink")
url = "1.ogg"
#filesink.set_property("location",url)
pipeline.add( autoaudiosrc)
pipeline.add( audioconvert)
pipeline.add( vorbisenc)
pipeline.add( oggmux)
pipeline.add( filesink)

autoaudiosrc.link( audioconvert)
audioconvert.link( vorbisenc)
vorbisenc.link( oggmux)
oggmux.link( filesink)

pipeline.set_state(Gst.State.PLAYING)
Gtk.main()

In [2]:

#!/usr/bin/env python
""" Source buf """
# -*- Mode: Python -*-
# vi:si:et:sw=4:sts=4:ts=4
#

import gobject
gobject.threads_init()

#import pygst
#pygst.require('0.10')
import gst

class Bufsrc(gst.BaseSrc):
    """ Push text """
    #here we register our plugin details
    __gstdetails__ = (
        "bufsrc test plugin",
        "bufsrc.py",
        "Source element that create a buffer",
        "Oleksandr Lavrushchenko <____@gmail.com>")
 
    _src_template = gst.PadTemplate ("src",
                                     gst.PAD_SRC,
                                     gst.PAD_ALWAYS,
                                     gst.caps_new_any ())
 
    __gsttemplates__ = (_src_template,)
 
    def __init__ (self, *args, **kwargs):
        gst.BaseSrc.__init__(self)
        gst.info('creating srcpad')
        self.src_pad = gst.Pad (self._src_template)
        self.src_pad.use_fixed_caps()

        #self.add_pad (self.src_pad)

    def do_create(self, offset, length):
        buf = gst.Buffer("data\n")
        buf.timestamp = 0
        buf.duration = pow(2, 63) -1
        return gst.FLOW_OK, buf

# Register element class
gobject.type_register(Bufsrc)
gst.element_register(Bufsrc, 'bufsrc', gst.RANK_MARGINAL)

sink = gst.element_factory_make("udpsink", "sink")
sink.set_property("port", 5000)
sink.set_property("host", "127.0.0.1")

pusher = Bufsrc()

player = gst.Pipeline("player")
player.add(pusher, sink)
gst.element_link_many(pusher2, sink)

player.set_state(gst.STATE_PLAYING)

gobject.MainLoop().run()

ImportError: No module named gst

In [15]:
%%writefile plugins/mysink.py
#!/usr/bin/env python
# -*- Mode: Python -*-
# vi:si:et:sw=4:sts=4:ts=4

# sinkelement.py
# (c) 2005 Edward Hervey <edward@fluendo.com>
# (c) 2007 Jan Schmidt <jan@fluendo.com>
# Licensed under LGPL
#
# Small test application to show how to write a sink element
# in 20 lines in python and place into the gstreamer registry
# so it can be autoplugged or used from parse_launch.
#
# You can run the example from the source doing from gst-python/:
#
#  $ export GST_PLUGIN_PATH=$GST_PLUGIN_PATH:$PWD/plugin:$PWD/examples/plugins
#  $ GST_DEBUG=python:4 gst-launch-1.0 fakesrc num-buffers=10 ! mysink

#gi.require_version('Gtk', '3.0')
import gi
gi.require_version('GstBase', '1.0')
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject, GstBase
Gst.init(None)

#
# Simple Sink element created entirely in python
#
class MySink(GstBase.BaseSink):
    __gstmetadata__ = ('CustomSink','Sink', \
                      'Custom test sink element', 'Edward Hervey')

    __gsttemplates__ = Gst.PadTemplate.new("sink",
                                           Gst.PadDirection.SINK,
                                           Gst.PadPresence.ALWAYS,
                                           Gst.Caps.new_any())

    def do_render(self, buffer):
        Gst.info("timestamp(buffer):%s" % (Gst.TIME_ARGS(buffer.pts)))
        return Gst.FlowReturn.OK

GObject.type_register(MySink)
__gstelementfactory__ = ("mysink", Gst.Rank.NONE, MySink)


Overwriting plugins/mysink.py


In [18]:
%%sh
export GST_PLUGIN_PATH=$GST_PLUGIN_PATH:$PWD/plugin
#:$PWD/examples/plugins 
GST_DEBUG=python:4 gst-launch-1.0 fakesrc num-buffers=10 ! mysink.py

In [17]:
!plugins/mysink.py

In [1]:
import sys, os
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject, Gtk

class GTK_Main(object):
    
    def __init__(self):
        window = Gtk.Window(Gtk.WindowType.TOPLEVEL)
        window.set_title("Vorbis-Player")
        window.set_default_size(500, 200)
        window.connect("destroy", Gtk.main_quit, "WM destroy")
        vbox = Gtk.VBox()
        window.add(vbox)
        self.entry = Gtk.Entry()
        vbox.pack_start(self.entry, False, False, 0)
        self.button = Gtk.Button("Start")
        vbox.add(self.button)
        self.button.connect("clicked", self.start_stop)
        window.show_all()
        
        self.player = Gst.Pipeline.new("player")
        source = Gst.ElementFactory.make("filesrc", "file-source")
        demuxer = Gst.ElementFactory.make("oggdemux", "demuxer")
        demuxer.connect("pad-added", self.demuxer_callback)
        self.audio_decoder = Gst.ElementFactory.make("vorbisdec", "vorbis-decoder")
        audioconv = Gst.ElementFactory.make("audioconvert", "converter")
        audiosink = Gst.ElementFactory.make("autoaudiosink", "audio-output")
        
        self.player.add(source)
        self.player.add(demuxer)
        self.player.add(self.audio_decoder)
        self.player.add(audioconv)
        self.player.add(audiosink)

        source.link(demuxer)
        self.audio_decoder.link(audioconv)
        audioconv.link(audiosink)
        
        bus = self.player.get_bus()
        bus.add_signal_watch()
        bus.connect("message", self.on_message)

    def start_stop(self, w):
        if self.button.get_label() == "Start":
            filepath = self.entry.get_text()
            if os.path.isfile(filepath):
                self.button.set_label("Stop")
                self.player.get_by_name("file-source").set_property("location", filepath)
                self.player.set_state(Gst.State.PLAYING)
        else:
            self.player.set_state(Gst.State.NULL)
            self.button.set_label("Start")

    def on_message(self, bus, message):
        t = message.type
        if t == Gst.MessageType.EOS:
            self.player.set_state(Gst.State.NULL)
            self.button.set_label("Start")
        elif t == Gst.MessageType.ERROR:
            err, debug = message.parse_error()
            print "Error: %s" % err, debug
            self.player.set_state(Gst.State.NULL)
            self.button.set_label("Start")
    
    def demuxer_callback(self, demuxer, pad):
        adec_pad = self.audio_decoder.get_static_pad("sink")
        pad.link(adec_pad)


GObject.threads_init()
Gst.init(None)        
GTK_Main()
Gtk.main()

-c:4: PyGIWarning: Gtk was imported without specifying a version first. Use gi.require_version('Gtk', '3.0') before import to ensure that the right version gets loaded.
-c:71: PyGIDeprecationWarning: Since version 3.11, calling threads_init is no longer needed. See: https://wiki.gnome.org/PyGObject/Threading
-c:9: PyGTKDeprecationWarning: Using positional arguments with the GObject constructor has been deprecated. Please specify keyword(s) for "type" or use a class specific constructor. See: https://wiki.gnome.org/PyGObject/InitializerDeprecations
-c:17: PyGTKDeprecationWarning: Using positional arguments with the GObject constructor has been deprecated. Please specify keyword(s) for "label" or use a class specific constructor. See: https://wiki.gnome.org/PyGObject/InitializerDeprecations
